# Imports

In [89]:
import pandas
import numpy

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# Load Data

In [90]:
path = '/content/drive/MyDrive/Colab Notebooks/QMIND_2023-2024/'
file_name = 'Hospital_Inpatient_Discharges__SPARCS_De-Identified___2019_20240203.csv'
df = pandas.read_csv(path + file_name, nrows=10000)
display(df)

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,...,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,Hudson Valley,Westchester,5902001.0,1045.0,White Plains Hospital Center,30 to 49,106,M,Other Race,Not Span/Hispanic,...,Moderate,Minor,Medical,Private Health Insurance,NaN,NaN,NaN,True,26507.00,4773.11
1,Hudson Valley,Westchester,5902001.0,1045.0,White Plains Hospital Center,70 or Older,105,F,Black/African American,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,NaN,NaN,NaN,True,20693.00,5631.30
2,Hudson Valley,Westchester,5902001.0,1045.0,White Plains Hospital Center,50 to 69,105,F,Other Race,Not Span/Hispanic,...,Extreme,Extreme,Surgical,Medicare,Blue Cross/Blue Shield,NaN,NaN,False,138252.00,29848.69
3,Hudson Valley,Westchester,5902001.0,1045.0,White Plains Hospital Center,50 to 69,106,F,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicare,Medicaid,NaN,NaN,True,11799.00,2595.72
4,Hudson Valley,Westchester,5902001.0,1045.0,White Plains Hospital Center,70 or Older,105,F,Other Race,Not Span/Hispanic,...,Moderate,Moderate,Medical,Medicare,Private Health Insurance,NaN,NaN,True,52711.00,11355.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,New York City,Kings,7001002.0,1286.0,Brookdale Hospital Medical Center,0 to 17,112,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,False,9239.36,5691.19
9996,New York City,Kings,7001002.0,1286.0,Brookdale Hospital Medical Center,0 to 17,112,M,Black/African American,Spanish/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,18037.28,11560.27
9997,New York City,Richmond,7004003.0,1740.0,Staten Island University Hosp-North,30 to 49,103,F,Other Race,Spanish/Hispanic,...,Moderate,Minor,Medical,Medicaid,Medicaid,NaN,NaN,True,7788.46,2085.42
9998,New York City,Richmond,7004003.0,1737.0,Staten Island University Hospital Prince's Bay,50 to 69,103,M,Black/African American,Not Span/Hispanic,...,Moderate,Minor,Medical,Medicaid,NaN,NaN,NaN,False,215554.33,135783.46


# Data Exploration

In [91]:
df.columns

Index(['Hospital Service Area', 'Hospital County',
       'Operating Certificate Number', 'Permanent Facility Id',
       'Facility Name', 'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition', 'Discharge Year', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'CCSR Procedure Code',
       'CCSR Procedure Description', 'APR DRG Code', 'APR DRG Description',
       'APR MDC Code', 'APR MDC Description', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Birth Weight',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

In [92]:
columns_to_keep = ['Age Group', 'Gender', 'Race', 'Ethnicity', 'Length of Stay',
                   'Type of Admission', 'APR MDC Description', 'APR Severity of Illness Description',
                   'APR Risk of Mortality', 'APR Medical Surgical Description']
df = df[columns_to_keep]
display(df)

,Age Group,Gender,Race,Ethnicity,Length of Stay,Type of Admission,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description
0,30 to 49,M,Other Race,Not Span/Hispanic,2,Emergency,INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR...,Moderate,Minor,Medical
1,70 or Older,F,Black/African American,Not Span/Hispanic,3,Emergency,DISEASES AND DISORDERS OF THE KIDNEY AND URINA...,Moderate,Moderate,Medical
2,50 to 69,F,Other Race,Not Span/Hispanic,7,Elective,DISEASES AND DISORDERS OF THE KIDNEY AND URINA...,Extreme,Extreme,Surgical
3,50 to 69,F,Black/African American,Not Span/Hispanic,1,Emergency,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,Moderate,Minor,Medical
4,70 or Older,F,Other Race,Not Span/Hispanic,3,Emergency,"INJURIES, POISONINGS AND TOXIC EFFECTS OF DRUGS",Moderate,Moderate,Medical
...,...,...,...,...,...,...,...,...,...,...
9995,0 to 17,M,Black/African American,Not Span/Hispanic,5,Emergency,MENTAL DISEASES AND DISORDERS,Moderate,Minor,Medical
9996,0 to 17,M,Black/African American,Spanish/Hispanic,14,Emergency,MENTAL DISEASES AND DISORDERS,Moderate,Minor,Medical
9997,30 to 49,F,Other Race,Spanish/Hispanic,1,Emergency,DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,Moderate,Minor,Medical
9998,50 to 69,M,Black/African American,Not Span/Hispanic,68,Elective,MENTAL DISEASES AND DISORDERS,Moderate,Minor,Medical


In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Age Group                            10000 non-null  object
 1   Gender                               10000 non-null  object
 2   Race                                 10000 non-null  object
 3   Ethnicity                            10000 non-null  object
 4   Length of Stay                       10000 non-null  object
 5   Type of Admission                    10000 non-null  object
 6   APR MDC Description                  10000 non-null  object
 7   APR Severity of Illness Description  9986 non-null   object
 8   APR Risk of Mortality                9986 non-null   object
 9   APR Medical Surgical Description     10000 non-null  object
dtypes: object(10)
memory usage: 781.4+ KB


In [94]:
df.describe(include='all')

,Age Group,Gender,Race,Ethnicity,Length of Stay,Type of Admission,APR MDC Description,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description
count,10000,10000,10000,10000,10000,10000,10000,9986,9986,10000
unique,5,2,4,4,79,6,26,4,4,3
top,50 to 69,F,White,Not Span/Hispanic,2,Emergency,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,Moderate,Minor,Medical
freq,2952,5487,5449,8170,2308,6794,1278,3737,5338,7302


In [95]:
for column in df.columns:
    print(column)
    print(df[column].value_counts(dropna=False))
    print()

Age Group
50 to 69       2952
70 or Older    2905
30 to 49       2061
0 to 17        1102
18 to 29        980
Name: Age Group, dtype: int64

Gender
F    5487
M    4513
Name: Gender, dtype: int64

Race
White                     5449
Other Race                2672
Black/African American    1795
Multi-racial                84
Name: Race, dtype: int64

Ethnicity
Not Span/Hispanic    8170
Spanish/Hispanic     1347
Unknown               458
Multi-ethnic           25
Name: Ethnicity, dtype: int64

Length of Stay
2      2308
3      1735
1      1667
4      1036
5       664
       ... 
103       1
67        1
66        1
69        1
54        1
Name: Length of Stay, Length: 79, dtype: int64

Type of Admission
Emergency        6794
Elective         2016
Newborn           585
Urgent            578
Trauma             26
Not Available       1
Name: Type of Admission, dtype: int64

APR MDC Description
DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM                                            1278
PRE

# Data Preparation/Processing

In [96]:
# Drop rows with NA/NULL values
df = df.dropna().reset_index(drop=True)
print(df.shape)

(9986, 10)


In [97]:
# Drop rows with infrequent values
df = df[~df['Type of Admission'].isin(['Not Available', 'Trauma'])]
df = df[df['Race'] != 'Multi-racial']
df = df[df['Ethnicity'] != 'Multi-ethnic']
df = df[~df['APR MDC Description'].isin(['MYELOPROLIFERATIVE DISEASES AND DISORDERS, AND POORLY DIFFERENTIATED NEOPLASM', 'DISEASES AND DISORDERS OF THE FEMALE REPRODUCTIVE SYSTEM', 'DISEASES AND DISORDERS OF THE MALE REPRODUCTIVE SYSTEM', 'HUMAN IMMUNODEFICIENCY VIRUS INFECTIONS', 'MULTIPLE SIGNIFICANT TRAUMA', 'DISEASES AND DISORDERS OF THE EYE', 'PRE MDC', 'BURNS'])]
df = df[df['APR Medical Surgical Description'] != 'Not Applicable']
print(df.shape)

(9561, 10)


In [ ]:
# Combine length of stay into categories (e.g., 1 day, 2 days, 3 days, 4 days, 5 days, 6 or more days)

In [ ]:
# Split into training and testing subsets

In [ ]:
# Min-max scale numeric features

In [ ]:
# One-hot encode categorical features

# Training

# Evaluation